In [ ]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import re
import openpyxl
import datetime
import os
import locale # es un set de variables de entorno que define el formato de fecha y hora, 
#el lenguaje, los encoding, el formato numérico, -entre otras cosas- del sistema
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')# Setea la variable LC_ALL al conjunto de código 
#UTF-8 con descripción español España

### SUBIR PADRON SSS

In [ ]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo =  archivo.name # es para obtner la direccion correcta del archivo con extencion.
        columnas = [    "Código de Obra Social",
                        "CUIT del empleador",
                        "CUIL del Titular",
                        "Código de Parentesco",
                        "Cuil",
                        "Tipo Documento",
                        "Número Documento",
                        "Apellido y Nombre",
                        "Sexo",
                        "Estado Civil",
                        "Fecha de Nacimiento",
                        "Nacionalidad",
                        "Calle",
                        "Número de puerta",
                        "Piso",
                        "Departamento",
                        "Localidad",
                        "Código Postal",
                        "Provincia",
                        "Tipo de Domicilio",
                        "Teléfono",
                        "Situación revista",
                        "Incapacidad",
                        "Tipo de beneficiario titular",
                        "Fecha de alta en la Obra Social",
                        "Fecha de cierre de la presentación",
                        "Valida CUIL",
                        "CUIL informado por la OS",
                        "Tipo beneficiario informado en la DDJJ (Declaración Jurada del empleador)",
                        "CUIT empleador informante en la DDJJ",
                        "Obra Social declarada en la DDJJ",
                        "Ultimo período declarado por el empleador en la DDJJ",
                        "Obra Social elegida (en caso de opción)",
                        "Período desde de vigencia de la opción"
                        ]
        try:
            padron_sss = pd.read_csv(ruta_archivo, sep='|', names=columnas)

            print (padron_sss)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

## cargar padron de sidefi

In [ ]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron = pd.read_excel(ruta_archivo)
            padron = pd.read_csv(archivo, sep=";", encoding="UTF-8")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

In [ ]:
padron.columns

###  transformar dependencia y Nr OS en sidefi 

In [23]:
padron['Dependencia']=padron['Dependencia'].astype(str)

dependencias = {
"10" : "CABA",
"20" : "ZONA NORTE Y NOROESTE",
"30" : "ZONA OESTE",
"40" : "ZONA SUR",
"50" : "LA PLATA",
"60" : "MAR DEL PLATA",
"70" : "PERGAMINO",
"80" : "ROSARIO",
"90" : "CORDOBA",
"100" : "MENDOZA",
"110" : "TUCUMAN"
}

convenios ={
    1 : "osoetsylra",
    4 : "SD",
    5 : "TIS",
    7 : "EVP",
    10: "MEPLIFE",
    12: "0405MEPLIFE",
    11: "osoetsylra",
    14: "osoetsylra",
    15: "osoetsylra"
}

padron['Dependencia']=padron['Dependencia'].replace(dependencias)
padron['Dependencia'].head(5)
padron['Nro. OS']=padron['Nro. OS'].replace(convenios)

# creo la columna desregulados para poder trabajarla con la redistribucion del padron x sedes.
padron['desregulados'] = padron['Nro. OS'].eq('osoetsylra')
padron['Nro. Documento'] = padron['Nro. Documento'].astype("Int64")



In [ ]:
padron['Nro. Documento']

## Cruzar los datos 

In [ ]:
cruce_sidefi = pd.merge(padron, padron_sss, left_on='Nro. Documento', right_on='Número Documento', how='outer', indicator=True)
cruce_sidefi
#save_path = filedialog.askdirectory()
#cruce_sidefi.to_excel(save_path + '\cruce_sss.xlsx', engine='openpyxl' ,index = False )
cruce_sidefi.columns

In [ ]:
cruce_sidefi['Tipo de beneficiario titular'].fillna(20, inplace=True)
cruce_sidefi['Tipo de beneficiario titular'].astype(int)
cruce_sidefi['desregulados'].head()


###  modelado tabla final

In [27]:
# crear nuevos campos codem y monotributo

cruce_sidefi["CODEM"] = ""
cruce_sidefi["MONOTRIBUTO SSS"] = ""

cruce_sidefi['Dependencia'].head(5)
tipo_benef= {
0:"RELACION DE DEPENDENCIA",
1:"PASANTES",
2:"JUBILADOS DEL SISTEMA NACIONAL DEL SEGURO DE SALUD",
3:"ADHERENTES VOLUNTARIOS",
4:"PEQUEÑOS CONTRIBUYENTES – MONOTRIBUTISTAS (LEY 25.865)",
5:"SERVICIO DOMESTICO (LEY 26.068)",
6:"JUBILADOS DEL REGISTRO PARA LA ATENCION DE ESPAÑOLES",
7:"MONOTRIBUTISTAS EFECTORES SOCIALES",
8:"BENEFICIARIOS DE SEGURO DE DESEMPLEO (LEY 24.013)",
9:"BENEFICIARIOS DE SEGURO DE DESEMPLEO (LEY 25.191) (RENATRE)",
10:"JUBILADOS Y PENSIONADOS DEL INSTITUTO NACIONAL DE SERVICIOS SOCIALES",
11:"JUBILADOS Y PENSIONADOS DE OTROS SISTEMAS",
12:"RELACION DE DEPENDENCIA (SIN APORTES Y CONTRIBUCION POR SJP)",
20:""
}
cruce_dic = {
    "left_only":"Sidefi",
    "both":"ambos",
    "right_only":"SSS"
    }
cruce_sidefi['_merge'].replace(cruce_dic, inplace=True)
cruce_sidefi.rename(columns={'_merge':'padron sidefi/sss'}, inplace= True)

cruce_sidefi['Tipo de beneficiario titular'].replace(tipo_benef, inplace=True)
cruce_sidefi['Tipo Afiliado'].replace(tipo_benef, inplace=True)

parentesco = {
0 : "titular",
1 : "conyuge",
2 : "concubino/a",
3 : "hijo soltero menor 21",
4 : "hijo soltero 21 a 25 cursando estudios",
5 : "hijo del conyuge soltero menor 21",
1 : "hijo del conyuge soltero 21 a 25 cursando estudios",
1 : "menor bajo guardia o tutela",
8 : "familiar a cargo",
9 : "mayor 25, discapacitado"
}

cruce_sidefi['Código de Parentesco'].replace(parentesco, inplace = True)

cruce_sidefi=cruce_sidefi[[
"Nro. OS",
"Plan del Afiliado",
"Parentesco con el TITULAR",
"Apellido Afiliado",
"Nombre Afiliado",
"Nro. Documento",
"Nro. CUIL",
"Fecha Nacimiento",
"Fecha Alta",
"Fecha Baja",
"Observacion Baja",
"Tipo Afiliado",
"Dependencia",
"Localidad_x",
"Provincia_x",
"CUIL del Titular",
"Código de Parentesco",
"Cuil",
"Número Documento",
"Apellido y Nombre",
"Sexo_y",
"Fecha de Nacimiento",
"Localidad_y",
"Código Postal",
"Provincia_y",
"Teléfono",
"Incapacidad",
"Tipo de beneficiario titular",
"Fecha de alta en la Obra Social",
"CUIL informado por la OS",
"Obra Social elegida (en caso de opción)",
"Período desde de vigencia de la opción",
"padron sidefi/sss",
"CODEM",
"MONOTRIBUTO SSS",
"desregulados"
]]

### redistribucion de padrones por sedes

In [28]:
ahora = datetime.datetime.now()    # Formatear la fecha y hora como una cadena con el formato "aaaa-mm-dd hh:mm:ss"
fecha = ahora.strftime("%Y-%m-%d %H-%M-%S")

groups = cruce_sidefi.groupby('Dependencia') # Agrupar el dataframe por la columna sede
dfs = [] # Crear una lista vacía para almacenar los dataframes separados
for name, group in groups: # Iterar sobre los grupos y añadir cada dataframe a la lista
    dfs.append(group)
    #print(name) # Imprimir la lista de dataframes
# Configurar los padrones que van a trabajar entre varias sedes 
#path = funciones.seleccionar_directorio()
#dfs[11].to_excel(path + '/' + "prueba.xlsx", index=False) # Guardar el dataframe en el archivo excel sin el índice
# 2: la plata, 0: caba , 1:cordoba , 3:mar del plata, 4: mendoza, 5:pegamino, 6:rosario, 7:tucuman
# 8: zona norte, 9: zona oeste, 10: zona sur

### <span style="color:pink">Distribución del padrón</span>
- **Mendoza** (+ SMT 60% desregulados)
- **BsAs - San Martín**


In [29]:
# San martin
dfs[8]=dfs[8].sort_values(by = "Nro. OS" , ascending = False) # ordeno el dataframe por convenio, para que no quden mezclados los responsables, y cuando se asignen los responsables tengan continuidad con los diff convenios
total_desregulado = dfs[8][dfs[8]['desregulados'] == False] # filtro los desregualdos "false" del df "dfs[8]" 
filas_mendoza = int(len(total_desregulado) * 0.6) #calculo el 60% de los "false" filtrados 
dfs[8]['asignado'] = 'vanesa' #creo una columna nueva con los valores "vanesa"
mendoza_indices = total_desregulado.iloc[:filas_mendoza].index # aca asigno el 60% al indice solo a los Falses
dfs[8].loc[mendoza_indices, 'asignado'] = 'Alejandro'

- **BsAs - Chile**
- **Rosario** (+ Chile Desregulados 100%)

In [30]:
#CABA
dfs[0]=dfs[0].sort_values(by = "Nro. OS" , ascending = False) # ordeno el dataframe por convenio, para que no quden mezclados los responsables, y cuando se asignen los responsables tengan continuidad con los diff convenios
total_desregulado = dfs[0][dfs[0]['desregulados'] == False] # filtro los desregualdos "false" del df "dfs[8]" 
filas_rosario = len(total_desregulado) #es el total de los "false" filtrados 
dfs[0]['asignado'] = 'Orne-Xime-Vero' #creo una columna nueva con los valores "vanesa"
rosario_indices = total_desregulado.iloc[:filas_rosario].index # aca asigno el 60% al indice solo a los Falses
dfs[0].loc[rosario_indices, 'asignado'] = 'Yamila-Neiva'

- **BsAs - San Justo**
- **BsAs - Pergamino** (+ SJT 35% desregulados)
- **Tucumán** (35% SJT Desregulados)

In [ ]:
print(dfs[9]["Nro. OS"])

In [32]:
# San Justo
dfs[9]=dfs[9].sort_values(by = "Nro. OS" , ascending = False) # ordeno el dataframe por convenio, para que no quden mezclados los responsables, y cuando se asignen los responsables tengan continuidad con los diff convenios
total_desregulado = dfs[9][dfs[9]['desregulados'] == False] # filtro los desregualdos "false" del df "dfs[8]" 
filas_pergamino = int(len(total_desregulado) * 0.35) #calculo el 60% de los "false" filtrados 
dfs[9]['asignado'] = 'Yanina' #creo una columna nueva con los valores "vanesa"
pergamino_indices = total_desregulado.iloc[:filas_pergamino].index # aca asigno el 60% al indice solo a los Falses
dfs[9].loc[pergamino_indices, 'asignado'] = 'Nancy'
#calcular el rango para san tucuman : claudia
inicio_tucuman = filas_pergamino
fin_tucuman = inicio_tucuman + filas_pergamino
# asignar este 35% a las filas de desregulados a claudia
tucuman_indices = total_desregulado.iloc[inicio_tucuman:fin_tucuman].index
dfs[9].loc[tucuman_indices,'asignado'] = 'Claudia'

- **BsAs - Quilmes** (+ La Plata)
- **Córdoba** (+ MDQ) (+ 40% desregulados Quilmes)
- **Mendoza** (+ 40% QM desregulados)

In [ ]:
# Quilmes
dfs[10]=dfs[10].sort_values(by = "Nro. OS" , ascending = False) # ordeno el dataframe por convenio, para que no quden mezclados los responsables, y cuando se asignen los responsables tengan continuidad con los diff convenios
total_desregulado = dfs[10][dfs[10]['desregulados'] == False] # filtro los desregualdos "false" del df "dfs[10]" 
filas_cordoba = int(len(total_desregulado) * 0.4) #calculo el 60% de los "false" filtrados 
dfs[10]['asignado'] = 'Micaela' #creo una columna nueva con los valores "vanesa"
cordoba_indices = total_desregulado.iloc[:filas_cordoba].index # aca asigno el 25% al indice solo a los Falses
dfs[10].loc[cordoba_indices, 'asignado'] = 'Maria Jose'
#calcular el rango para san tucuman : claudia
inicio_mendoza = filas_cordoba
fin_mendoza = inicio_mendoza + filas_cordoba
# asignar este 35% a las filas de desregulados a claudia
mendoza_indices = total_desregulado.iloc[inicio_mendoza:fin_mendoza].index
dfs[10].loc[mendoza_indices,'asignado'] = 'ALejandro'

filas_cordoba


In [34]:
dfs[0].to_excel('prueba_znorte.xlsx')

### guardar el excel por  sede

In [ ]:
ahora = datetime.datetime.now()    # Formatear la fecha y hora como una cadena con el formato "aaaa-mm-dd hh:mm:ss"
mes = ahora.strftime("%B").capitalize()
dia = ahora.day  # Obtener el día en número
anio = ahora.year  # Obtener el año en número
fecha = f"{mes} {dia} {anio}"

save_path = filedialog.askdirectory()
cruce_sidefi.to_excel(save_path + '\devoluciones2_septiembre2023.xlsx', engine='openpyxl' ,index = False )

# Definir funciones de estilo para cada rango de columnas
def highlight_col1(columns):
    return ['background-color: #33ffff' if col != '' else '' for col in columns]

def highlight_col2(columns):
    return ['background-color: #74FC69' if col != '' else '' for col in columns]


### guardarlos con los nombres correspondientes
for df in dfs: # Iterar sobre la lista de dataframes
    sede = df['Dependencia'].iloc[0] # Obtener el nombre de la sede del primer elemento del grupo 
    #df.drop('desregulados', axis = 1, inplace=True)
    styled_df = df.style.apply(highlight_col1, subset=df.columns[0:15])  # Aplicar a las columnas de 'A' a 'E'
    styled_df = styled_df.apply(highlight_col2, subset=df.columns[15:32])  # Aplicar a las columnas de 'F' a 'J'
    directorio = os.path.join(save_path, sede, sede + " " + fecha)
    os.makedirs(directorio, exist_ok=True)
    filename = os.path.join(directorio, sede+ " " + fecha + '.xlsx')
    #filename = directorio + '/' + sede + '.xlsx' # Crear el nombre del archivo con la ruta de la carpeta y el nombre del convenio
    styled_df.to_excel(filename, index=False) # Guardar el dataframe en el archivo excel sin el índice
    print(f"Se ha guardado el archivo {filename}") # Imprimir un mensaje de confirmación